In [0]:
# INCLUDE_HEADER_TRUE
# INCLUDE_FOOTER_TRUE

# Automating identity management

In this lab you will learn how to:
* Remotely administer account level users and groups using the SCIM API

##Prerequisites

If you would like to follow along with this lab, you will need account administrator capabilities over your Databricks account.

## Overview

Databricks exposes all its major functionality as a collection of REST APIs. In this lab, we will focus on using the SCIM API to remotely create and delete users, service principals and groups.

There are a number of ways to apply usage of this API:

* Implement code that uses a low level web access API (like Python's **requests** module for example) to issue REST calls and interpret the results
* Use a client that provides low level web access (like **curl**, **wget** or **Postman**) to issue calls and view the results
* Integrate a higher level automation framework that supports SCIM. SCIM is an open standard implemented by many identity provider frameworks. For general info on SCIM, refer to the <a href="http://www.simplecloud.info/" target="_blank">SCIM website</a>.

Regardless of which approach you take to hook into your Databricks account using SCIM, you need the following:
* The URL of the Databricks SCIM interface. This is the top-level URL on which all API endpoints are based.
* A SCIM token for authenticating with the server.

Depending on the operation, you may need to specify additional data to fulfill the request.

## Setup

In order to invoke the Databricks account SCIM API, we need:
* To enable user provisioning
* The SCIM API URL, which includes your Databricks account URL and account ID
* A SCIM token

Run the following cell to create a landing zone for the needed inputs, then follow the instructions below.

In [0]:
dbutils.widgets.text(name='url', defaultValue='')
dbutils.widgets.text(name='token', defaultValue='')

import os
os.environ["DBACADEMY_SCIM_TOKEN"] = f"Authorization: Bearer {dbutils.widgets.get('token')}"
os.environ["DBACADEMY_SCIM_URL"] = dbutils.widgets.get('url')

Now let's populate the two fields as follows.

1. In the account console, click **Settings** in the left sidebar.
1. Select the **User provisioning** tab.
1. Enable user provisioning, if it isn't already enabled.
1. Click **Regenerate token**, copy the resulting token to the clipboard, click **Done** and paste the token to the *token* field.
1. Copy the value for **Account SCIM URL** into the *url* field

Pasting these values into their associated fields will automatically trigger the previous cell, which will populate OS environment variables that will be used by the commands throughout this lab.

## Users and service principals

In the lab *Managing Account Identities*, we saw how to manage users and service prinicpals using the account console. Let's perform a similar set of tasks using the SCIM API. For more info, refer to the <a href="https://docs.databricks.com/dev-tools/api/latest/scim/account-scim.html" target="_blank">Account SCIM API documentation</a>.

### Querying users
Let's get a list of users in the account.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Users" | json_pp

We can get an expanded view of a specific user. From the output above, identify a user and copy the value for *id*. Substitute that value for *ID* in the following cell and run it.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Users/ID" | json_pp

### Creating a user
Let 's add a new user to our account. For this we need to **`POST`** to the same endpoint that we queried earlier. We'll need to specify JSON data describing the new user; at a minimum, a *userName*. Using a combination of shell features, we are inlining the JSON data below the command itself.

For the purposes of this training exercise, I am using a temporary email address courtesy of <a href="https://www.dispostable.com/" target="_blank">dispostable.com</a>. When following along, feel free to use an email address of your choosing.

In [0]:
%sh cat << EOF | curl -s -X POST -H "${DBACADEMY_SCIM_TOKEN}" -H "Content-type: text/json" "${DBACADEMY_SCIM_URL}/Users" -d @- | json_pp
{
  "userName": "dbanalyst0906_curl@dispostable.com"
}
EOF

As we would expect when creating a user in the account console, the new user will be issued an email inviting them to join and set their password.

As a reminder, this operation creates an identity at the account level only. This user will not be able to access Databricks services yet since they have not been assigned to any workspaces.

### Querying service principals
Let's get a list of service principals in the account.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/ServicePrincipals" | json_pp

We can get a view of a specific service principal. From the output above, identify a service principal and copy the value for *id* (not *applicationId*). Substitute that value for *ID* in the following cell and run it.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/ServicePrincipals/ID" | json_pp

### Creating a service principal
Let 's add a new service principal to our account. For this we need to **`POST`** to the same endpoint that we queried earlier. We need to specify JSON data describing the new service principal; at a minimum, a *displayName*. To do this we'll apply the same pattern we used earlier.

In [0]:
%sh cat << EOF | curl -s -X POST -H "${DBACADEMY_SCIM_TOKEN}" -H "Content-type: text/json" "${DBACADEMY_SCIM_URL}/ServicePrincipals" -d @- | json_pp
{
  "displayName": "service_principal_curl"
}
EOF

## Groups

Now that we know the basics of querying and creating users and service principals, let's turn our attention to groups.

### Querying groups
Let's get a list of groups in the account.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Groups" | json_pp

We can get a view of a specific group. From the output above, identify a group and copy the value for *id*. Substitute that value for *ID* in the following cell and run it.

In [0]:
%sh curl -s -X GET -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Groups/ID" | json_pp

### Creating a group
Let 's add a new group to our account. For this we need to **`POST`** to the same endpoint that we queried earlier. We'll need to specify JSON data describing the new group; at a minimum, a *displayName*. To do this we'll apply the same pattern we used earlier.

In [0]:
%sh cat << EOF | curl -s -X POST -H "${DBACADEMY_SCIM_TOKEN}" -H "Content-type: text/json" "${DBACADEMY_SCIM_URL}/Groups" -d @- | json_pp
{
  "displayName": "group_curl"
}
EOF

This creates an empty group. The API allows you to specify members at create time, but we'll see how to do this in a separate call now.

### Adding group members
Let's add a the user and service principal we created earlier to this new group. For this we need to **`PATCH`** the group-specific endpoint. The JSON data required to support this operation is a little more complex than the previous examples, and we'll need to perform three substitutions before executing the following cell:
* Replace *GROUP* with the *id* value from the group creation output
* Replace *USER* with the *id* value from the user creation output
* Replace *SERVICEPRINCIPAL* with the *id* value from the service principal creation output

In [0]:
%sh cat << EOF | curl -s -X PATCH -H "${DBACADEMY_SCIM_TOKEN}" -H "Content-type: text/json" "${DBACADEMY_SCIM_URL}/Groups/GROUP" -d @- | json_pp
{
  "Operations": [
    {
      "op": "add",
      "value": {
        "members": [
          {
            "value": "USER"
          },
          {
            "value": "SERVICEPRINCIPAL"
          }
        ]
      }
    }
  ]
}
EOF

### Removing group members
Removing members from a group is done with a similar **`PATCH`** operation, but the JSON syntax is different. Let's see this in action by removing the service principal from the group. We'll need to perform two substitutions before executing the following cell:
* Replace *GROUP* with the *id* value from the group creation output
* Replace *SERVICEPRINCIPAL* with the *id* value from the service principal creation output

In [0]:
%sh cat << EOF | curl -s -X PATCH -H "${DBACADEMY_SCIM_TOKEN}" -H "Content-type: text/json" "${DBACADEMY_SCIM_URL}/Groups/GROUP" -d @- | json_pp
{
  "Operations": [
    {
      "op": "remove",
      "path": "members[value eq \"SERVICEPRINCIPAL\"]"
    }
  ]
}
EOF

## Cleanup

Let's now explore how to remove principals by cleaning up those that we created in this lab.

### Deleting groups
Let's delete the group we created. For this we use **`DELETE`** against the same endpoint that we used earlier for adding and removing members. Replace *GROUP* with the *id* value from the group creation output.

Note that this only deletes the group but leaves its members behind. We will delete those explicitly.

In [0]:
%sh curl -s -X DELETE -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Groups/GROUP" 

### Deleting users
Let's delete the user we created. Again we use **`DELETE`** against the same endpoint we'd use to query a user. Replace *USER* with the *id* value from the user creation output.

In [0]:
%sh curl -s -X DELETE -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/Users/USER" 

### Deleting service principals
Finally, let's delete the service principal we created. Again we use **`DELETE`** against the same endpoint we'd use to query a service principal. Replace *SERVICEPRINCIPAL* with the *id* value from the service principal creation output.

In [0]:
%sh curl -s -X DELETE -H "${DBACADEMY_SCIM_TOKEN}" "${DBACADEMY_SCIM_URL}/ServicePrincipals/SERVICEPRINCIPAL" 